Title

In [22]:
import sys
import os
import pandas as pd
from data_utils import load

In [23]:
from openai import OpenAI

# Initialize client with your API key
client = OpenAI(api_key=openai.api_key)

# Send a message to ChatGPT
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, can you explain Python classes?"}
    ]
)

# Print the response
print(response.choices[0].message.content)


NameError: name 'openai' is not defined

In [16]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
print("-")

-


In [11]:
import psutil

# Total RAM in GB
total_ram = psutil.virtual_memory().total / (1024**3)
print(f"Total RAM: {total_ram:.2f} GB")

Total RAM: 31.43 GB


In [22]:
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

In [24]:
from transformers import BlipProcessor, BlipForConditionalGeneration

models = [
    "Salesforce/blip-image-captioning-base",
    "Salesforce/blip2-flan-t5-xl",
    "Salesforce/blip2-opt-2.7b"
]

for name in models:
    try:
        print(f"Testing {name}...")
        processor = BlipProcessor.from_pretrained(name, local_files_only=True)
        model = BlipForConditionalGeneration.from_pretrained(name, local_files_only=True)
        print(f" {name} loaded successfully.\n")
    except Exception as e:
        print(f" {name} failed: {e}\n")

Testing Salesforce/blip-image-captioning-base...


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


 Salesforce/blip-image-captioning-base loaded successfully.

Testing Salesforce/blip2-flan-t5-xl...


You are using a model of type blip-2 to instantiate a model of type blip. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


 Salesforce/blip2-flan-t5-xl failed: Error(s) in loading state_dict for BlipVisionEmbeddings:
	size mismatch for class_embedding: copying a param with shape torch.Size([1, 1, 1408]) from checkpoint, the shape in current model is torch.Size([1, 1, 768]).

Testing Salesforce/blip2-opt-2.7b...
 Salesforce/blip2-opt-2.7b failed: the JSON object must be str, bytes or bytearray, not NoneType



In [14]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# 1️⃣ Load processor and model
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")

# Select device (CPU in this case)
device = "cpu"
model.to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/1.jpg"
image = Image.open(img_path).convert("RGB")

# Prepare inputs
prompt = "Identify tobacco brands in this picture."
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

# Generate caption
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)

# Decode and print caption
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generated caption: cigarette lighters


In [18]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

# Load BLIP-2 small (faster, less memory)
processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xl")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

# Generate caption
prompt = "Describe any tobacco products in this image."
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model.generate(**inputs)

caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Caption:", caption)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Caption: cigarettes


In [26]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, infer_device

model_id = "IDEA-Research/grounding-dino-tiny"
device = infer_device()

processor = AutoProcessor.from_pretrained(model_id)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_id).to(device)

# Load an image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")
text_labels = [["a cat", "a remote control"]]

inputs = processor(images=image, text=text_labels, return_tensors="pt").to(model.device)
with torch.no_grad():
    outputs = model(**inputs)

In [30]:
from transformers import AutoImageProcessor, DetaForObjectDetection
from PIL import Image
import requests

img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

image_processor = AutoImageProcessor.from_pretrained("jozhang97/deta-swin-large")
model = DetaForObjectDetection.from_pretrained("jozhang97/deta-swin-large")

inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to Pascal VOC format (xmin, ymin, xmax, ymax)
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.2, target_sizes=target_sizes)[
    0
]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

Detected car with confidence 0.835 at location [1.32, 1118.76, 1226.91, 1830.19]
Detected person with confidence 0.822 at location [1186.63, 932.25, 1388.06, 1438.77]
Detected person with confidence 0.815 at location [592.0, 786.17, 738.98, 1182.0]
Detected car with confidence 0.586 at location [2097.77, 1041.53, 2766.14, 1831.14]
Detected bench with confidence 0.532 at location [1876.75, 950.63, 2419.83, 1186.25]
Detected backpack with confidence 0.503 at location [625.07, 851.05, 707.43, 983.36]
Detected person with confidence 0.443 at location [2211.33, 874.89, 2353.19, 1070.74]
Detected person with confidence 0.363 at location [94.89, 991.79, 137.52, 1125.23]
Detected person with confidence 0.239 at location [2224.38, 875.44, 2352.0, 955.89]
Detected backpack with confidence 0.216 at location [1233.98, 1016.32, 1352.48, 1153.64]
Detected tv with confidence 0.203 at location [1110.09, 661.41, 1727.29, 931.18]
Detected bench with confidence 0.202 at location [1885.31, 952.4, 2414.69,

In [18]:
from transformers import pipeline
from PIL import Image
import requests

# Load the zero-shot object detection pipeline with the specified model
detector = pipeline("zero-shot-object-detection", model="iSEE-Laboratory/llmdet_large")

# Define the candidate classes (tobacco-related items)
candidate_classes = ["cigarette", "vape", "lighter", "cigar", "tobacco pouch","smoke"]

# Load an example image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/2.jpg"
image = Image.open(img_path).convert("RGB")

# Perform zero-shot object detection
results = detector(image, candidate_labels=candidate_classes, threshold=0.5)

print("Detected objects:")
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.3f}")

Device set to use cpu


Detected objects:
Label: cigar, Score: 0.607
Label: smoke, Score: 0.505


In [21]:
print("Detected objects:")
print(results)
for result in results:
    print(f"Label: {result['label']}, Score: {result['score']:.3f}")
print(image.size) 

Detected objects:
[{'score': 0.6067025661468506, 'label': 'cigar', 'box': {'xmin': 1287, 'ymin': 976, 'xmax': 1332, 'ymax': 1010}}, {'score': 0.5047281980514526, 'label': 'smoke', 'box': {'xmin': 1288, 'ymin': 884, 'xmax': 1659, 'ymax': 1019}}]
Label: cigar, Score: 0.607
Label: smoke, Score: 0.505
(2767, 1847)


In [36]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import torch

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b")

device = "cpu"  # force CPU
model.to(device)

image = Image.open("data/Images_test-20240424T090736Z-001/Images_test/1.jpg").convert("RGB")
prompt = "Question: Describe any tobacco products in this image."

inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

# Generate caption
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=50)  # limit tokens

caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)

OSError: Can't load image processor for 'Salesforce/Salesforce/blip2-flan-t5-small'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Salesforce/Salesforce/blip2-flan-t5-small' is the correct path to a directory containing a preprocessor_config.json file

In [30]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

# Load processor and model (image-captioning base)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Load image
img_path = "data/Images_test-20240424T090736Z-001/Images_test/1.jpg"
image = Image.open(img_path).convert("RGB")

# Prepare inputs
inputs = processor(images=image, return_tensors="pt")

# Generate caption
with torch.no_grad():
    outputs = model.generate(**inputs)

# Decode and print
caption = processor.decode(outputs[0], skip_special_tokens=True)
print("Generated caption:", caption)

Generated caption: a cup of coffee and a box of cigarettes
